In [235]:
import pandas as pd
import numpy as np
import re
import json

In [236]:
pd.set_option('display.max_columns',50)

In [237]:
dataFile = './speciesdata.csv'

In [238]:
excludedCols = ['Responsável','Guia','Folder','Artigo','Família.1', 
                'Glossário','Fotos adultos','Fotos girinos',
                'Vídeos','Áudios', 'Diagnose','DOI' ]

df = pd.read_csv(dataFile, usecols=lambda x: x not in excludedCols)

In [239]:
# Drop every row after the one containing only null values
nullrow = df[df.apply(lambda x: all(x.isnull()), axis=1)].index[0]
df.drop(axis=1, index=range( nullrow,df.shape[0] ), inplace=True)

In [240]:
df.drop('Red List.1', axis=1, inplace=True)

In [241]:
df.rename( {'Espécie': 'speciesName', 
            'Família': 'family',
            'Postura dos ovos (habitat breeding)*': 'habitat_breeding',
            'Chances de encontros (muito raro/raro/frequente/muito frequente)': 'detectability',
            'Nome Comum': 'vernacularNames'}, axis=1, inplace=True)

In [242]:
df.replace('^-$', '', inplace=True, regex=True)

In [243]:
df

,family,speciesName,vernacularNames,Endêmico (Chapada),Endêmico (Cerrado),Red List,Diagnose escrita,Habitat de vida (mata ou floresta/area aberta/pedras)*,Categorias fitofisionomias (guia),Altitude,habitat_breeding,Poleiro (tipical calling perch)*,Atividade (noturno/diurno),Tamanho real (mm) - fêmea,Tamanho real (mm) - macho,Meses de Ocorrência,detectability,Distribuição - SITE REUBER (espécies do DF),Ameaças - SITE REUBER (espécies do DF)
0,AROMOBATIDAE,"Allobates goianus (Bokermann, 1975)",,não,sim,NT,ok,Floresta: mata de galeria com ambiente rochoso,FG,NaN,terrestre (Tr),Terrestre (Te),diurno,16.8-17,16.8-17,NaN,Muito raro,,
1,BUFONIDAE,"Rhaebo guttatus (Schneider, 1799)",Cururu de couro,não,não,LC,ok,Mata de galeria e cerradão,"FG, CE",NaN,Lêntico (Le),Terrestre (Te),noturno,180,68,NaN,Raro,,
2,BUFONIDAE,"Rhinella mirandaribeiroi (Gallardo, 1965)",Cururu,não,sim,não avaliada,ok,cerrado (Ce) e areas alteradas (Aa),"CSS, AA",NaN,Lêntico (Le),Terrestre (Te),noturno,49-73,40-71,NaN,Raro,,
3,BUFONIDAE,"Rhinella ocellata (Günther, 1858)",Cururu,não,não,LC,NaN,"cerrado (Ce) e areas alteradas (Aa), mata ciliar","FC, CSS, AA",200-1500m,terrestre (Tr),Terrestre (Te),noturno,44,NaN,chuva até março,Frequente,"Tocantins, Para´, Mato Grosso, Minas Gerais, a...",
4,BUFONIDAE,"Rhinella rubescens (Lutz, 1925)*",Cururu-vermelho,não,sim,LC,ok,"areas abertas (Of), Cerrado (Ce), floresta (Fh...","FC, FG, CE, CSS, C, V, AA",430-1400 m,Lêntico (Le) e Lótico (Lo),Terrestre (Te),noturno,90,NaN,ano todo,Frequente,Espécie com ampla distribuição associada ao bi...,Desmatamentos e remoção de hábitats constituem...
5,BUFONIDAE,"Rhinella schneideri (Werner, 1894)*",Sapo cururu,não,não,LC,ok,"areas abertas (Of), Cerrado (Ce), floresta (Fh...","CSS, C, V, AA",NaN,Lêntico (Le) e Lótico (Lo),Terrestre (Te),noturno,150-200,NaN,ano todo,Muito frequente,Espécie com ampla distribuição podendo ser enc...,Espécie não ameaçada.
6,CRAUGASTORIDAE,"Barycholos ternetzi (Miranda- Ribeiro, 1937)*",Rãzinha da chuva,não,sim,LC,ok,"floresta (Fh), cerrado, (Ce)","FC, FG, CE,CSS",NaN,terrestre (Tr),Terrestre (Te),diurno,23-30,NaN,chuva,Frequente,"Espécie restrita ao bioma Cerrado, pode ser en...",Desmatamento e remoção de hábitats constituem ...
7,DENDROBATIDAE,"Ameerega flavopicta (Lutz, 1925)*",Sapo flecha,não,sim,LC,ok,"areas abertas (Of), Cerrado (Ce)","SCR, C, CR",NaN,terrestre (Tr),Terrestre (Te),diurno,20-24,NaN,ano todo,Raro,Espécie com ampla distribuição no bioma Cerrad...,Desmatamentos e remoção de habitat.
8,HYLIDAE,"Aplastodiscus lutzorum (Berneck, Giaretta, Bra...",Perereca-verde-da-mata,não,sim,não avaliada,ok,floresta (Fh),"FC, FG",NaN,Lótico (Lo),Arbustos baixos (Hb) e arbustos altos (Lb),noturno,33.7,30.7–36,Chuva (Dezembro a Março),Raro,"Sua distribuição é no estado do DF e no Goiás,...",
9,HYLIDAE,"Bokermannohyla pseudopseudis (Miranda-Ribeiro,...",Falsa perereca,não,sim,LC,ok,"areas abertas (Of), Cerrado (Ce) e floresta (Fh)","CR, FG, FC",NaN,Lótico (Lo),Terrestre (Te) e arbustos baixos (Lb),noturno,60.2-61.6,51.9-65.3,chuva,Frequente,,


### Species name, id and Authorship

In [244]:
df['scientificName'] = df['speciesName'].apply(lambda x: re.findall( '^\w+(?: cf.| aff.)? \w+\.?', str(x))[0])

In [245]:
df['scientificNameAuthorship'] = df[['speciesName','scientificName']].apply(lambda x: x[0][len(x[1]):], axis=1)
df['scientificNameAuthorship'] = df['scientificNameAuthorship'].apply(lambda x: re.findall( '[\w,\s\-&]+', str(x) )).apply(lambda x: ''.join(x).strip())

In [246]:
ids = df['scientificName'].str.replace('\s','-').str.replace('[.]','').str.lower()
df.insert(0, 'id', ids)

In [247]:
df.drop('speciesName', axis=1, inplace=True)

### Family name and id

In [248]:
df['family'] = df['family'].apply(lambda x: x.capitalize())
df['family_id'] = df['family'].str.lower()

### Vernacular name

In [249]:
def extractVernacularNames(string):
    names = re.split('/|;|,',string)
    names = [ n.strip().replace(' ','-').capitalize() for n in names ]
    if len(names)==1 and names[0]=='':
        return []
    else:
        return names

In [250]:
df['vernacularNames'] = df['vernacularNames'].apply(extractVernacularNames)

### Red list

In [251]:
df['Red List'] = df['Red List'].str.lower()
df.loc[ df['Red List']=='não avaliada','Red List']='ne'

In [252]:
df = pd.get_dummies(data=df, prefix='redlist', columns=['Red List'])

### Endemicidade

In [253]:
df['endemic_cerrado'] = df['Endêmico (Cerrado)'].apply(lambda x: 1 if x=='sim' else 0)
df['endemic_chapada'] = df['Endêmico (Chapada)'].apply(lambda x: 1 if x=='sim' else 0)
df.drop('Endêmico (Chapada)', axis=1, inplace=True)
df.drop('Endêmico (Cerrado)', axis=1, inplace=True)

In [254]:
# Helpers

getInsideParentheses = lambda x: [ str.lower(e) for e in re.findall( '\((.{0,5})\)', str(x) ) ]

### Detectability

In [255]:
df['detectability'] = df['detectability']\
    .str.replace('[^\w]','')\
    .str.replace('ê','e')\
    .str.lower()

In [256]:
subst_dict = {
    'muitofrequente': 'ff',
    'frequente':'f',
    'raro': 'r',
    'muitoraro': 'rr'
}

df['detectability'] = df['detectability'].apply(lambda x: subst_dict[x] if x is not np.NaN else x)
df = pd.get_dummies(df, prefix='detectability',columns=['detectability'])

### Poleiro

In [257]:
df['poleiro'] = df['Poleiro (tipical calling perch)* '].apply( getInsideParentheses )
df.drop('Poleiro (tipical calling perch)* ', axis=1, inplace=True)

In [258]:
poleiro_types = list(set( el for ls in df['poleiro'] for el in ls))

In [259]:
for tp in poleiro_types:
    df[f'tcp_{tp}'] = df['poleiro'].apply(lambda x: 1 if f'{tp}' in x else 0)

In [260]:
df.drop('poleiro', axis=1, inplace=True)

### Habitat breeding

In [261]:
matches = df['habitat_breeding'].str.lower().str.extractall('\((\w+)\)')
hb_dummies = pd.get_dummies(matches, prefix="habitat_breeding").groupby(level=0).sum()

In [262]:
hb_dummies = hb_dummies.reindex(df.index).fillna(0).astype(int)

In [263]:
df = pd.concat([df,hb_dummies], axis=1)

In [264]:
df.drop('habitat_breeding',axis=1, inplace=True)

### Phytophysiognomies categoricals

In [265]:
df.rename({'Categorias fitofisionomias (guia)': 'phytos'}, axis=1, inplace=True)

In [266]:
replaces = {
    'cdd':'css',
    'fg':'mg',
    'fc':'mc',
    'ce':'cc',
    'css':'ce',
    'sc':'cet',
    'scr':'cer',
    'c': 'cal',
    'cr':'car',
    'aa':'ur',
    'v':'ve'
} 
df['phytos'] = df['phytos'].str.lower().str.strip().str.split('\s*,\s*')\
    .apply(lambda x: [replaces.get(i,i) for i in x] if isinstance(x,list) else '')\
    .str.join(',')

In [267]:
df['phytos']

0                        mg
1                     mg,cc
2                     ce,ur
3                  mc,ce,ur
4     mc,mg,cc,ce,cal,ve,ur
5              ce,cal,ve,ur
6               mc,mg,cc,ce
7               cer,cal,car
8                     mc,mg
9                 car,mg,mc
10                   cal,ce
11                cal,ce,ur
12                cal,ce,ur
13                cal,ce,ur
14                 mc,mg,ce
15                    mc,mg
16                mc,mg,cal
17                    mc,mg
18             cal,mc,mg,ur
19                         
20         cal,css,mc,mg,ur
21                   cal,ce
22                   cal,ce
23                cal,ce,ur
24          cal,ce,mc,mg,ur
25                      cal
26          cal,ce,mg,mc,ur
27             cal,ce,mc,mg
28                   cal,ce
29               cal,ce,car
30                 mc,mg,ur
31                   cal,ce
32                   cal,ce
33                   cal,ce
34                      cal
35       cal,ce,ur,m

### Habitat

In [268]:
df['habitat'] = df['Habitat de vida (mata ou floresta/area aberta/pedras)*'].apply( getInsideParentheses)
df.drop('Habitat de vida (mata ou floresta/area aberta/pedras)*', axis=1, inplace=True)

In [269]:
habitat_types = list(set( el for ls in df['habitat'] for el in ls ))

In [270]:
for tp in habitat_types:
    df[f'habitat_{tp}'] = df['habitat'].apply(lambda x: 1 if f'{tp}' in x else 0)

In [271]:
df.drop('habitat', axis=1, inplace=True)

### Atividade

In [272]:
df['Atividade (noturno/diurno)'].value_counts()

noturno              44
noturno e diurno      4
diurno                3
noturno e diurno      1
Name: Atividade (noturno/diurno), dtype: int64

In [273]:
df['atividade_diu'] = df['Atividade (noturno/diurno)'].apply(lambda x: 1 if 'diurno' in str(x) else 0)
df['atividade_not'] = df['Atividade (noturno/diurno)'].apply(lambda x: 1 if 'noturno' in str(x) else 0)
df.drop('Atividade (noturno/diurno)', axis=1, inplace=True)

In [274]:
df['Ameaças - SITE REUBER (espécies do DF)'][5]

'Espécie não ameaçada.'

### Tamanho

In [275]:
df['tamanho_femea'] = df['Tamanho real (mm) - fêmea'].apply(lambda x: re.findall( '([0-9\.]+)' , str(x).replace(',','.')))
df['tamanho_macho'] = df['Tamanho real (mm) - macho'].apply(lambda x: re.findall( '([0-9\.]+)' , str(x).replace(',','.')))

In [276]:
df['tamanho_femea_med'] = df['tamanho_femea'].apply( lambda l: sum([float(i) for i in l])/len(l) if len(l)>0 else None )
df['tamanho_macho_med'] = df['tamanho_macho'].apply( lambda l: sum([float(i) for i in l])/len(l) if len(l)>0 else None )

In [277]:
#df['tamanho_femea_min'] = df['tamanho_femea'].apply( lambda x: min([ float(i) for i in x]) if len(x) > 0 else None)
#df['tamanho_femea_max'] = df['tamanho_femea'].apply( lambda x: max([ float(i) for i in x]) if len(x) > 0 else None)
#df['tamanho_macho_min'] = df['tamanho_macho'].apply( lambda x: min([ float(i) for i in x]) if len(x) > 0 else None)
#df['tamanho_macho_max'] = df['tamanho_macho'].apply( lambda x: max([ float(i) for i in x]) if len(x) > 0 else None)

#df['tamanho_macho_max'].replace(np.NaN, df['tamanho_femea_max'], inplace=True)
#df['tamanho_macho_min'].replace(np.NaN, df['tamanho_femea_min'], inplace=True)
#df['tamanho_femea_max'].replace(np.NaN, df['tamanho_macho_max'], inplace=True)
#df['tamanho_femea_min'].replace(np.NaN, df['tamanho_femea_min'], inplace=True)

In [278]:
df.drop('tamanho_femea', axis=1, inplace=True)
df.drop('tamanho_macho', axis=1, inplace=True)
df.drop('Tamanho real (mm) - fêmea', axis=1, inplace=True)
df.drop('Tamanho real (mm) - macho', axis=1, inplace=True)

In [279]:
df['tamanho_macho_med'].replace(np.NaN, df['tamanho_femea_med'], inplace=True)
df['tamanho_femea_med'].replace(np.NaN, df['tamanho_macho_med'], inplace=True)
df['tamanho_med'] = df[['tamanho_macho_med','tamanho_femea_med']].mean(axis=1)

### Meses de ocorrência

1. Extract month ranges from strings

In [280]:
import unicodedata

normalize_str = lambda x: unicodedata.normalize('NFKD',x).encode('ascii', errors='ignore').decode('utf-8').lower().strip()
months=['janeiro','fevereiro','marco','abril','maio','junho','julho','agosto','setembro','outubro','novembro','dezembro',
        'jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']

In [281]:
df['Meses de Ocorrência'] = df['Meses de Ocorrência'].apply(lambda x: normalize_str(x) if pd.notnull(x) else x)\
    .replace('chuva','out-mar')\
    .replace('ano todo', 'jan-dez')\
    .replace('seca-chuva','jan-dez')\
    .replace('chuva ate marco','out-mar')\
    .str.findall("("+"|".join(months)+")")\
    .apply(lambda x: [ i[:3] for i in x] if isinstance(x,list) else x)\
    .str.join(sep='-')\

In [282]:
def occurrence_months(m_start, m_end):
    d=['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
    idx_mstart = d.index(m_start)
    idx_mend = d.index(m_end)
    
    if idx_mstart > idx_mend:
        return d[idx_mstart:]+d[:idx_mend+1]
    else:
        return d[idx_mstart:idx_mend+1]
    
    
def months_to_array(mths):
    d=['jan','fev','mar','abr','mai','jun','jul','ago','set','out','nov','dez']
    return [ 1 if i in mths else 0 for i in d ]

In [283]:
df['month_vec'] = df['Meses de Ocorrência'].str.split('-').apply(
    lambda x: months_to_array(occurrence_months(*x)) if isinstance(x,list) else [0 for i in range(12)]
)

### Temporarily remove unused fields

In [284]:
df.drop('Distribuição - SITE REUBER (espécies do DF)', axis=1, inplace=True)
df.drop('Ameaças - SITE REUBER (espécies do DF)', axis=1, inplace=True)

### Remove unwanted species

In [285]:
spp_to_remove = ['boana-sp','leptodactylus-aff-cunicularius','elachistocleis-sp']

In [286]:
df = df[ df['id'].apply(lambda x: x not in spp_to_remove) ]

### Sort dataframe by Family and id

In [287]:
df = df.sort_values(by=['family','id'])

### Write to json

In [288]:
species_data_path = '../_data/species.json'

d = df.to_dict(orient='records')

with open(species_data_path, 'w') as f:
    json.dump(d, f,indent=1, ensure_ascii=False)